In [1]:
import xarray as xr

from dask.distributed import Client, progress
from dask_kubernetes import KubeCluster

In [2]:
cluster = KubeCluster()
cluster.scale(4);
cluster

In [3]:
cluster.adapt(maximum=)
client = Client(cluster)

In [4]:
pangeo = 'ESIP-AWS-S3'

In [5]:
if pangeo=='ESIP-AWS-S3':
    import s3fs
    fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False)
    fileObj = fs.open('esip-pangeo/pangeo/adcirc/adcirc_01.nc')

In [6]:
if pangeo=='PANGEO-GCS-S3':
    import gcsfs
    fs = gcsfs.GCSFileSystem(project='pangeo-181919')
    fileObj = fs.open('pangeo-data/rsignell/adcirc_01.nc')

In [7]:
ds = xr.open_dataset(fileObj, engine='h5netcdf', chunks={'time':10, 'node':141973})

In [8]:
ds['zeta']

<xarray.DataArray 'zeta' (time: 720, node: 9228245)>
dask.array<shape=(720, 9228245), dtype=float64, chunksize=(10, 141973)>
Coordinates:
  * time     (time) datetime64[ns] 2031-08-03T02:10:00 ... 2031-08-08T02:00:00
    x        (node) float64 dask.array<shape=(9228245,), chunksize=(141973,)>
    y        (node) float64 dask.array<shape=(9228245,), chunksize=(141973,)>
Dimensions without coordinates: node
Attributes:
    location:       node
    long_name:      water surface elevation above geoid
    mesh:           adcirc_mesh
    standard_name:  sea_surface_height_above_geoid
    units:          m

In [9]:
ds['zeta'].nbytes/1.e9

53.1546912

In [11]:
%%time
max_var = ds['zeta'].max(dim='time').compute()

CPU times: user 2min 10s, sys: 8.3 s, total: 2min 18s
Wall time: 14min 53s
